## 2.1 Shortest Path
- The transition propabiltiy matrix $N$ can be computed from $M$ via devision of the columns by the column sum
  $N_{i,j} = M_{i,j} / \sum_{k = 0}^{99}{M_{k,j}}$
- The most likely path is the sequence of $D$ rooms $x_i$ in $X$ that maximizes $N_{0,x_1}\cdot\prod_{i=2}^{D-1}{N_{x_i,x_{i+1}}} \cdot N_{x_D, 99}$

In [1]:
import numpy as np
import random
M = np.load("maze.npy")

In [27]:
# test with an arbitray matrix
M = np.zeros([100,100])
for i in range(1000):
    a = random.randint(0,99)
    b = random.randint(0, 99)
    if a != b:
        M[a,b] = 1
        M[b,a] = 1

In [42]:
# compute the transition probability matrix M
N = np.zeros([100, 100])
for i in range(100):
    N[:,i] = M[:,i] / np.sum(M[:,i])

In [20]:
# compute the shortest path, broad search
reached = np.zeros([100])
reached[0] = 1
last = [0]
while reached[99] == 0 and len(last) > 0:
    news = []
    for n in last:
        l = [i for i in range(100) if reached[i] == 0 and M[i, n] == 1]
        for i in l:
            reached[i] = n +2
        news.extend(l)
    last = news

In [21]:
path = [99]
pos = 99
while pos != 0:
    pos = reached[pos] - 2
    path.append(int(pos))
print "The (or one of the) fastest path from room 0 to room 99 is:"
print path[::-1]
print "that visits %i rooms in total" % len(path)

The (or one of the) fastest path from room 0 to room 99 is:
[0, 1, 4, 78, 77, 80, 83, 84, 85, 89, 91, 94, 96, 99]
that visits 14 rooms in total


In [64]:
# find the most likely path from 0 to 99
reached = np.zeros([100,2])
reached[0,:] = [1, 1.]
last = [0]
while len(last) > 0:
    news = set()
    for l in last:
        p = reached[l,1]
        p_n = 1. / np.sum(M[:,l]) *p
        n = [i for i in range(100) if M[i,l] == 1 and p_n > reached[i,1]]
        for n_ in n:
            reached[n_,0] = l + 2
            reached[n_,1]= p_n
        news = news.union(n)
    last = news

In [66]:
path = [99]
pos = 99
while pos != 0:
    pos = reached[pos,0] - 2
    path.append(int(pos))
print "The (or one of the) most likely path from room 0 to room 99 is:"
print path[::-1]
print "with probability %s" % reached[99, 1]

The (or one of the) most likely path from room 0 to room 99 is:
[0, 1, 4, 78, 79, 80, 83, 84, 85, 89, 91, 94, 96, 99]
With probability 4.42885487528e-10


##2.2 Expexted traversal time

The transition matrix $N$ was generated from the room Matrix $M$. We now define $N'$ which is equal to $N$ but in column $j$ which is all zeros (every walker that enters room $j$ disappears -> we do not need to track it anymore.
Further, the distribution of walker at step $t$ that have not yet entered room $j$ is given as $p_t$ with $p_0$ being all zeros but $p_{0,i} = 1$
The distribution after each step is now given as $p_t = N' \times p_{t-1} = (N')^t \times p_0$.
The average time $t_a$ the walker needs to reach room $j$ from room $i$ is then given as $t_a = \sum_{t=1}^\infty t \cdot p_{t,j}$

In [53]:
START = 0
END = 99

time_average = 0
dist = np.zeros([100])
dist[START] = 1
transitions = np.copy(N)
transitions[:,END] = 0
for t in range(1,100000): #upper time limit from which the average time does not change anymore (theoretically numberical analysis needed)
    dist = np.dot(transitions,dist)
    time_average += t * dist[END]
print "The expected average time from room %i to room %i is %f time steps" % (START, END, time_average)

The expected average time from room 0 to room 99 is 2712.520579 time steps


In [60]:
NUM_WALKS = 100000
cummulated_time = 0.

next_rooms = [[room2 for room2 in range(100) if M[room2, room] == 1] for room in range(100)]
for i in range(NUM_WALKS):
    room = START
    t = 0
    while room != END:
        t += 1
        room = random.choice(next_rooms[room])
    cummulated_time += t
print "The average time from room %i to room %i observed was %f time steps" % (START, END, cummulated_time / NUM_WALKS)

The average time from room 0 to room 99 observed was 2707.583360 time steps
